<a href="https://colab.research.google.com/github/Tryus111/Sell_car_project/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 25 13:31:46 2023

@author: user
"""
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#import file
df_cleaned = pd.read_csv('/train.csv')
# Preprocessing for deep learning
df_raw = df_cleaned.copy()

# Clean Unknown Data
df_raw = df_raw[df_raw['engine_size'] != 0]
df_raw = df_raw[df_raw['fuel_type'] != 'Unknown']
df_raw = df_raw[df_raw['drivetrain'] !='Unknown']

# Price
df_raw['price'] = pd.to_numeric(df_raw['price'], errors='coerce')
df_raw['price'] = df_raw['price'].astype(float)

# Change inf value to valid value
df_raw = df_raw.replace([np.inf, -np.inf], np.nan)
df_raw = df_raw.dropna()

from scipy.sparse import csr_matrix
# One Hot Encoding
index_column = [0, 1, 4, 6, 8, 9, 33, 34]
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),index_column)], remainder = 'passthrough')
df_raw = ct.fit_transform(df_raw)

if isinstance(df_raw, csr_matrix):
    df_raw = df_raw.toarray()

df_preprocessing = pd.DataFrame(df_raw, columns=ct.get_feature_names_out())

#Standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
unscaled_input = df_preprocessing.iloc[:, : -1]
targets = df_preprocessing.iloc[:, -1]
targets
unscaled_input_2d = unscaled_input.values.reshape(-1, 1)

scaled_input = sc.fit_transform(unscaled_input_2d)

# Shuffle data
shuffle_data = np.arange(scaled_input.shape[0])
np.random.shuffle(shuffle_data)

shuffle_inputs = scaled_input[shuffle_data]
shuffle_targets = targets[shuffle_data]

sample_count = shuffle_inputs.shape[0]

# Split data
training_sample = int(0.8 * sample_count)
validation_sample = int(0.1 * sample_count)
test_sample = sample_count - training_sample - validation_sample

train_inputs = shuffle_inputs[: training_sample]
train_targets = shuffle_targets[: training_sample]

validation_inputs = shuffle_inputs[training_sample : training_sample + validation_sample]
validation_targets = shuffle_targets[training_sample: training_sample + validation_sample]

test_inputs = shuffle_inputs[training_sample:]
test_targets = shuffle_targets[training_sample:]

# Save to npz
np.savez('training_data.npz', inputs = train_inputs, targets = train_targets)
np.savez('validation_data.npz', inputs = validation_inputs, targets = validation_targets)
np.savez('test_data.npz', inputs = test_inputs, targets = test_targets)


# Bagian Baru

# Bagian Baru